In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, size, avg, expr
import json

# spark = SparkSession.builder.appName("SummaryAnalysis").getOrCreate()

spark = SparkSession\
        .builder\
        .master("spark://192.168.2.229:7077") \
        .appName("test")\
        .config("spark.dynamicAllocation.enabled", "true")\
        .config("spark.dynamicAllocation.shuffleTracking.enabled", "true")\
        .config("spark.shuffle.service.enabled", "false")\
        .config("spark.dynamicAllocation.executorIdleTimeout", "300s")\
        .config("spark.driver.port", "8888")\
        .config("spark.blockManager.port", "10005")\
        .config("spark.executor.cores", "2")\
        .getOrCreate()

def process_row(row):
    content_len = row.content_len
    summary_len = row.summary_len
    relative_len = content_len / summary_len
    decreased_len = content_len - summary_len
    percentage = summary_len / content_len
    return {
        #'content': row.content,
        #'summary': row.summary,
        'content_len': content_len,
        'summary_len': summary_len,
        'relative_len': relative_len,
        'decreased_len': decreased_len,
        'percentage': percentage
    }


# json_df = spark.read.json('volume/corpus-webis-tldr-17.json')
json_df = spark.read.json('test.json')

processed_df = json_df.rdd.map(process_row)

processed_df.map(json.dumps).saveAsTextFile('test_preprocessed_dataset.json')

# Stop the SparkSession
spark.stop()

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.